In [154]:
import numpy as np
import itertools
from skimage import color, io

In [155]:
def make_blocks_vectorized(x,d):
    return list(itertools.chain.from_iterable(np.hsplit(r, d) for r in np.split(x, d)))

In [156]:
S = 512 # original width and height of image
K = 64 # width and height of each piece

original_image = color.rgb2lab(io.imread("../data/sources/1200.png"))
original_squares = make_blocks_vectorized(original_image, S // K)
scrambled_image =  color.rgb2lab(io.imread("../data/1200.png"))
scrambled_squares = make_blocks_vectorized(scrambled_image, S // K)

# scrambled pieces
x_0 = scrambled_squares[0]
x_1 = scrambled_squares[1]

# correct pieces
x_2 = original_squares[0]
x_3 =original_squares[1]

In [157]:
x_0[0][63][0]

35.677094646907676

In [158]:
def dissimilarity(x_i, x_j, r='r'):
    return sum(sum((x_i[k][K-1][d] - x_j[k][0][d])**2 for d in range(3)) for k in range(K))

In [159]:
def vectorised_dissimilarity(x_i, x_j, r='r'):
    return np.sum(np.square(x_i[:,K-1] - x_j[:,0]))

In [160]:
dissimilarity(x_0, x_1)

50035.60969747558

In [161]:
vectorised_dissimilarity(x_0, x_1)

50035.60969747558

In [162]:
dissimilarity(x_2, x_3)

538.4051935376021

In [163]:
vectorised_dissimilarity(x_2, x_3)

538.4051935376021

In [164]:
def l_q_dissimiliarity(x_i, x_j, r='r', p=3/10, q=1/16):
    return np.power(sum(sum(np.power(np.absolute(x_i[k][K-1][d] - x_j[k][0][d]), p) for d in range(3)) for k in range(K)), q / p)

In [165]:
def vectorised_l_q_dissimiliarity(x_i, x_j, r='r', p=3/10, q=1/16):
    return np.power(np.sum(np.power(np.abs(x_i[:,K-1] - x_j[:,0]), p)), q/p)

In [166]:
l_q_dissimiliarity(x_0, x_1)

3.3933564063719466

In [167]:
vectorised_l_q_dissimiliarity(x_0, x_1)

3.3933564063719466

In [168]:
l_q_dissimiliarity(x_2, x_3)

2.929677477440781

In [169]:
vectorised_l_q_dissimiliarity(x_2, x_3)

2.929677477440781

In [170]:
def precalculate_dissimiliarities(D, parts, r='r'):
    dissimilarity_matrix = np.empty((len(parts), len(parts)))
    for i, x_i in enumerate(parts):
        for j, x_j in enumerate(parts):
            if i == j:
                continue
            dissimilarity_matrix[i][j] = D(x_i, x_j, r)
    return dissimilarity_matrix

In [171]:
scrambled_r_dissimilarity_matrix = precalculate_dissimiliarities(vectorised_l_q_dissimiliarity, scrambled_squares)

In [172]:
original_r_dissimilarity_matrix = precalculate_dissimiliarities(vectorised_l_q_dissimiliarity, original_squares)

In [173]:
def normalised_compatibility(dissimilarities, i, j):
    return np.exp(-dissimilarities[i][j]/np.percentile(dissimilarities[i], 25))

In [177]:
normalised_compatibility(scrambled_r_dissimilarity_matrix, 1, 2)

0.3527559890857335

In [178]:
normalised_compatibility(original_r_dissimilarity_matrix, 1, 2)

0.40427476716517763